# Expression utils demo

In [1]:
from wc_lang import SpeciesType, Species, Function, Observable, Parameter
from wc_lang.expression_utils import WcLangExpression, WcLangExpressionError, TokCodes

objects = {
            Species: {'test_id[c]':Species(), 'x_id[c]':Species()},
            Parameter: {'test_id':Parameter(), 'param_id':Parameter()},
            Observable: {'test_id':Observable(), 'obs_id':Observable()},
            Function: {'fun_1':Function(), 'fun_2':Function()}
        }

def make_wc_lang_expr(expr, obj_type=Function):
    objs = {}
    for o_type, value in objects.items():
        if o_type.__name__ in obj_type.Meta.valid_used_models:
            objs[o_type] = value
    return WcLangExpression(obj_type, 'expr_attr', expr, objs)


## Illustrate Python tokens

In [2]:
wc_lang_expr = make_wc_lang_expr('3 * 4 + identifier')
for token in wc_lang_expr.tokens:
    print(token)

TokenInfo(type=2 (NUMBER), string='3', start=(1, 0), end=(1, 1), line='3 * 4 + identifier')
TokenInfo(type=53 (OP), string='*', start=(1, 2), end=(1, 3), line='3 * 4 + identifier')
TokenInfo(type=2 (NUMBER), string='4', start=(1, 4), end=(1, 5), line='3 * 4 + identifier')
TokenInfo(type=53 (OP), string='+', start=(1, 6), end=(1, 7), line='3 * 4 + identifier')
TokenInfo(type=1 (NAME), string='identifier', start=(1, 8), end=(1, 18), line='3 * 4 + identifier')


## tokenize()

In [3]:
expr_2 = make_wc_lang_expr('2 * fun_1() + obs_id')
expr_2.tokenize()
assert expr_2.errors == []

## Illustrate WC-Lang tokens

In [4]:
print(TokCodes.__doc__)
for tok_code in TokCodes:
    print(tok_code.name)

 Token codes used in parsed expressions 
wc_lang_obj_id
math_fun_id
other


In [5]:
for wc_token in expr_2.wc_tokens:
    print(wc_token)

WcLangToken(tok_code=<TokCodes.other: 3>, token_string='2', model_type=None, model_id=None, model=None)
WcLangToken(tok_code=<TokCodes.other: 3>, token_string='*', model_type=None, model_id=None, model=None)
WcLangToken(tok_code=<TokCodes.wc_lang_obj_id: 1>, token_string='fun_1', model_type=<class 'wc_lang.core.Function'>, model_id='fun_1', model=<wc_lang.core.Function object at 0x7f03f809c0f0>)
WcLangToken(tok_code=<TokCodes.other: 3>, token_string='(', model_type=None, model_id=None, model=None)
WcLangToken(tok_code=<TokCodes.other: 3>, token_string=')', model_type=None, model_id=None, model=None)
WcLangToken(tok_code=<TokCodes.other: 3>, token_string='+', model_type=None, model_id=None, model=None)
WcLangToken(tok_code=<TokCodes.wc_lang_obj_id: 1>, token_string='obs_id', model_type=<class 'wc_lang.core.Observable'>, model_id='obs_id', model=<wc_lang.core.Observable object at 0x7f03f802de80>)


## eval()

In [6]:
expr_2.test_eval_expr()

3.0

## Disambiguate

In [7]:
expr_3 = make_wc_lang_expr('test_id + obs_id')
expr_3.tokenize()
print(expr_3.errors)

["'test_id + obs_id', a Function.expr_attr, contains multiple model object id matches: 'test_id' as a Observable id, 'test_id' as a Parameter id"]


In [8]:
expr_4 = make_wc_lang_expr('Observable.test_id + Parameter.test_id + obs_id')
expr_4.tokenize()
assert expr_4.errors == []
expr_4.test_eval_expr(test_val=2.0)

6.0